# L6: Multi-agent Collaboration for Financial Analysis


The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)
#processes could be sequential --loss of context--see documentation

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst. They have the skills and expertise to handle this task effectively.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL", "context": "Use statistical modeling and machine learning to identify trends and predict market movements for AAPL."}


> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL to analyze trends and predict movements accurately.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/AAPL"} 

Apple Inc. (AAPL) Stock Price, News,

Final Answer: 
Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
AAPL Apple Inc. 204.49 +0.28% Summary News Research Chart Community Statistics Historical Data Profile Financials Analysis Options Holders Sustainability NasdaqGS - Delayed Quote • USD
Apple Inc. (AAPL)
Follow
Add holdings
Time to buy AAPL?
203.92 +3.29 +(1.64%)
At close: June 6 at 4:00:01 PM EDT
204.49 +0.57 +(0.28%)
After hours: June 6 at 7:59:54 PM EDT
Time to buy AAPL?
1D
5D
1.53%
1M
2.73%
6M
-16.03%
YTD
-18.57%
1Y
4.85%
5Y
146.06%
All
158,780.55%

Key Events
Mountain

Advanced Chart
Loading Chart for AAPL

Previous Close
200.63
Open
202.99
Bid
203.80 x 1400
Ask
204.17 x 500
Day's Range
202.05 - 205.70
52 Week Range
169.21 - 260.10
Volume
46,121,557
Avg. Volume
61,982,165
Market Cap (intraday)
3.046T
Beta (5Y Monthly)
1.21
PE Ratio (TTM)
31.76
EPS (TTM)
6.42
Earnings Date
Jul 30, 2025 - Aug 4, 2025
Forward Dividend & Yield
1.04 (0.51%)
Ex-Dividend Date
May 12, 2025
1y Target Est
228.85

Apple Inc. O

Final Answer: 

Insights and alerts about significant market opportunities or threats for AAPL can be derived from the data analysis done by the Data Analyst. The market data for Apple Inc. (AAPL) shows a YTD return of -18.57%, a 1-year return of 4.85%, and a 5-year return of 146.06%. The company's profit margin is 24.30%, with a return on assets of 23.81% and a return on equity of 138.01%. The company's market cap is 3.046T, with a trailing P/E ratio of 31.76 and a forward P/E ratio of 25.45. The Data Analyst will further analyze the financial highlights, balance sheet, cash flow, and research insights to provide a comprehensive overview of market trends and potential opportunities for AAPL.

> Finished chain.
 [DEBUG]: [Crew Manager] Task output: Insights and alerts about significant market opportunities or threats for AAPL can be derived from the data analysis done by the Data Analyst. The market data for Apple Inc. (AAPL) shows a YTD return of -18.57%, a 1-year return of 4.85%, and

I need to gather more information from my team members to create detailed execution plans for the trading strategies for AAPL.

Action: Ask question to co-worker
Action Input: {"coworker": "Trading Strategy Developer", "question": "Can you provide specific entry and exit points for each of the developed trading strategies for AAPL?", "context": "I need detailed execution plans for the strategies considering current market conditions and optimal pricing."}

> Entering new CrewAgentExecutor chain...
I need to gather specific entry and exit points for each trading strategy for AAPL. I should focus on current market conditions and optimal pricing to provide the best execution plans.

Action: Search the internet
Action Input: {"search_query": "AAPL trading strategy entry and exit points"} 


Search results: Title: Apple stock analysis: key indicators and charting tips for beginners
Link: https://medium.com/@topforextrade/apple-stock-analysis-key-indicators-and-charting-tips-for-beginners-36

I need to ensure a thorough risk analysis is conducted for the proposed trading strategies for AAPL to provide comprehensive mitigation recommendations.

Action: Delegate work to co-worker
Action Input: {"coworker": "Risk Advisor", "task": "Conduct a detailed risk analysis for the proposed trading strategies for AAPL and provide mitigation recommendations", "context": "Detailed execution plans for the trading strategies for AAPL"}


> Entering new CrewAgentExecutor chain...
I need to gather information on the proposed trading strategies for AAPL and conduct a thorough risk analysis to provide accurate mitigation recommendations.

Action: Search the internet
Action Input: {"search_query": "trading strategies for AAPL"} 


Search results: Title: AAPL Option Strategy Payout Diagram Apple - Market Chameleon
Link: https://marketchameleon.com/Overview/AAPL/StrategyPayout/
Snippet: Analyze Apple (AAPL) stock option trading strategies. Display payout diagrams showing gains and losses for Strad

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Dear Team,

I have conducted a thorough risk analysis for the proposed trading strategies for AAPL based on the detailed execution plans provided. After considering factors such as market volatility, company fundamentals, technical analysis, and risk tolerance, I have identified potential risks and developed mitigation recommendations for each strategy:

1. Momentum Trading Strategy:
- Potential Risks: The risk of significant losses if the trade goes against the momentum, sudden changes in market conditions affecting short-term momentum.
- Mitigation Recommendations: Set stop-loss orders to protect against losses, closely monitor market conditions for any signs of change in momentum.

2. Support and Resistance Strategy:
- Potential Risks: The risk of losses if the stock does not hold at key support levels, sudden shifts in market sentiment impacting resistance levels.
- Mitigation Recommendations: Utilize trailing stop-loss orders to secure profits and limit losses, conduct thorough technical analysis to confirm key support and resistance levels.

3. Trend Following Strategy:
- Potential Risks: The risk of losses if the trend reverses suddenly, failure to reach predefined profit targets.
- Mitigation Recommendations: Implement a trailing stop-loss strategy to protect profits, closely monitor trend indicators to identify potential reversals.

Overall, it is crucial to continuously assess and manage risks associated with the trading strategies for AAPL by staying informed about market dynamics, setting clear risk management guidelines, and adapting to changing conditions. By following the mitigation recommendations provided, we can optimize trading performance and minimize potential losses.

Thank you for your attention to this matter.

Sincerely,
[Your Name]
Crew Manager